In [9]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedShuffleSplit
from tabulate import tabulate

import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn

In [23]:
df = pd.read_csv("../cancer.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [25]:
df.drop(['Unnamed: 32', 'id'], axis=1, inplace=True)

## M - Malignant; B - Benign

Malignant are encoded as 0 and Benign are encoded as 1

In [26]:
df.diagnosis.value_counts(normalize=True)

diagnosis
B    0.627417
M    0.372583
Name: proportion, dtype: float64

In [27]:
encoder = LabelEncoder()
encoded_diagnosis = encoder.fit_transform(df['diagnosis'])
encoded_diagnosis = pd.DataFrame(encoded_diagnosis, columns=['diagnosis_encoded'])

In [28]:
encoded_df = pd.concat([df, encoded_diagnosis], axis=1)
encoded_df.drop('diagnosis', axis=1, inplace=True)
encoded_df.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,diagnosis_encoded
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [29]:
encoded_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   radius_mean              569 non-null    float64
 1   texture_mean             569 non-null    float64
 2   perimeter_mean           569 non-null    float64
 3   area_mean                569 non-null    float64
 4   smoothness_mean          569 non-null    float64
 5   compactness_mean         569 non-null    float64
 6   concavity_mean           569 non-null    float64
 7   concave points_mean      569 non-null    float64
 8   symmetry_mean            569 non-null    float64
 9   fractal_dimension_mean   569 non-null    float64
 10  radius_se                569 non-null    float64
 11  texture_se               569 non-null    float64
 12  perimeter_se             569 non-null    float64
 13  area_se                  569 non-null    float64
 14  smoothness_se            5

In [31]:
X = encoded_df.drop('diagnosis_encoded', axis=1)
y = encoded_df['diagnosis_encoded']

In [33]:
strat_shuf_split = StratifiedShuffleSplit(n_splits=1, test_size=.3, random_state=24)
train_idx, test_idx = next(strat_shuf_split.split(X, y))

In [36]:
X_train = X.iloc[train_idx]
y_train = y.iloc[train_idx]

X_test = X.iloc[test_idx]
y_test = y.iloc[test_idx]

In [37]:
X_train.shape

(398, 30)

In [38]:
y_train.shape

(398,)

In [41]:
nb = GaussianNB()

nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
score = accuracy_score(y_test, y_pred)
print(f"R2 score: {score}")

R2 score: 0.935672514619883


In [55]:
splits = [i/10 for i in range(1, 10)]
scores = []
results = []
nb = GaussianNB()

for i, split in enumerate(splits):
    strat_shuf_split = StratifiedShuffleSplit(n_splits=1, test_size=split, random_state=24)
    train_idx, test_idx = next(strat_shuf_split.split(X, y))
    
    X_train = X.iloc[train_idx]
    y_train = y.iloc[train_idx]

    X_test = X.iloc[test_idx]
    y_test = y.iloc[test_idx]

    nb.fit(X_train, y_train)
    y_pred = nb.predict(X_test)

    score = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    cr = classification_report(y_test, y_pred)

    results.append([i+1, str(f"{np.ceil((1-split)*100)}%"), str(f"{np.ceil(split*100)}%"), score])
    scores.append([i+1, str(f"{np.ceil((1-split)*100)}%"), str(f"{np.ceil(split*100)}%"), score, cr, cm])

headers = ['SNo.', 'Train %', 'Test %', 'Accuracy', 'Report', 'Confusion Matrix']
print(tabulate(scores, headers=headers, tablefmt='grid'))

+--------+-----------+----------+------------+-------------------------------------------------------+--------------------+
|   SNo. | Train %   | Test %   |   Accuracy | Report                                                | Confusion Matrix   |
+========+===========+==========+============+=======================================================+====================+
|      1 | 90.0%     | 10.0%    |   0.929825 | precision    recall  f1-score   support               | [[34  2]           |
|        |           |          |            |                                                       |  [ 2 19]]          |
|        |           |          |            |            0       0.94      0.94      0.94        36 |                    |
|        |           |          |            |            1       0.90      0.90      0.90        21 |                    |
|        |           |          |            |                                                       |                    |
|       

In [56]:
import csv
new_headers = ['SNo.', 'Train %', 'Test %', 'Accuracy']
with open("cancer_results.csv", 'w', newline='') as file:
    csvWriter = csv.writer(file)
    csvWriter.writerow(new_headers)
    csvWriter.writerows(results)